# Composite prime powers

Intention here is to analyze how the graph of primes with edges between prime numbers $u$, $v$ defined as

$$u\sim v \Leftrightarrow |u-v| = 2^\alpha 3^\beta \;\; \alpha,\beta \in \mathbb{N}$$

leads to some remarkable graph patterns.

We'll look at five different cases in function of allowed values for $\alpha$ and $\beta$:

- G: the values for $\alpha$ and $\beta$ are arbitrary positive integers
- $G_{00}$: the values for $\alpha$ and $\beta$ are not prime
- $G_{10}$: the values for $\alpha$ is prime and $\beta$ is not prime
- $G_{01}$: the values for $\alpha$ is not prime and $\beta$ is prime
- $G_{11}$: the values for $\alpha$ is prime and $\beta$ is prime

## Preliminaries

Some function and setup used in what follows.

In [60]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
 

In [32]:
def is_prime(n): 
    """Python function for checking prime or not"""
    if n <= 1:
        return False
    for i in range(2, n):
        if (n % i == 0):
            return False;
    return True

We'll use the first 5000 primes:

In [35]:
p = []
n = 2
M = 5000  #for first M number of primes
while (len(p) < M):
    if(is_prime(n)==True):
        p = p + [n] #p stores only prime numbers
    n = n +1 

The 5000x5000 set of edge endpoints:

In [36]:
from itertools import product
p1 = set(p)
n1 = list(set(product(p1, repeat = 2))) # n1 contains all possible prime tuples upto 5000

To speed up the computation we cache here some values. Specifically, the set of distances which constrain the allowed edges.

In [46]:

exponent_max = 10 # the highest exponent allowed
prime_exponents =np.array([u for u in range(exponent_max) if is_prime(u)])
non_prime_exponents = np.array([u for u in range(exponent_max) if not is_prime(u)])
any_exponent = range(exponent_max)
def create_distances(alpha:bool, beta:bool):
    """Generates a list of distances with the given constraints on the exponents."""
    p = prime_exponents if alpha==True else non_prime_exponents
    q = prime_exponents if beta==True else non_prime_exponents
    distance_tuples = list(set(product(set(pow(2,p)),set(pow(3,q)))))
    distances = [ np.prod(u) for u in distance_tuples]
    return distances

The construction of the full graph is now:

In [61]:
def create_full_graph():
    distances =  [ np.prod(u) for u in list(set(product(set(pow(2,np.arange(exponent_max))),set(pow(3,np.arange(exponent_max))))))]
    valid_endpoints = [(u,v) for u,v in n1 if np.abs(u-v) in distances]
    g = nx.Graph()
    g.add_edges_from(valid_endpoints)
    return g
    
G = create_full_graph()

The constraint sub-graphs on the other hand:

In [62]:
def create_constrained_graph(alpha, beta):
    distances = create_distances(alpha,beta);
    valid_endpoints = [(u,v) for u,v in n1 if np.abs(u-v) in distances]
    g = nx.Graph()
    g.add_edges_from(valid_endpoints)
    return g

G00 = create_constrained_graph(False, False)
G10 = create_constrained_graph(True, False)
G01 = create_constrained_graph(False, True)
G11 = create_constrained_graph(True, True)

## Connected components

The unconstrained graph has one large component with around 70K edges:

In [66]:
C = [G.subgraph(c).copy() for c in sorted(nx.connected_components(G), key=len, reverse=True)]
print(f"Components: {len(C)}, nodes: {len(G.nodes)}, edges: {len(G.edges)} ") # It gives number of connected components for case 1

Components: 1, nodes: 5000, edges: 70437 


For the constrained cases we get the following:

In [71]:
for name in ["G00","G10","G01","G11"]:
    g = vars()[name]
    c = [g.subgraph(c).copy() for c in sorted(nx.connected_components(g), key=len, reverse=True)]
    print(f"{name}: components: {len(c)}, nodes: {len(g.nodes)}, edges: {len(g.edges)} ")

G00: components: 1, nodes: 4999, edges: 18655 
G10: components: 2, nodes: 4999, edges: 17010 
G01: components: 6, nodes: 4999, edges: 17828 
G11: components: 12, nodes: 4997, edges: 16944 


The reason that the  node count does not equal to 5000 is due to the way the graph is constructed from the edge tuples. It means that G11 has effectively three isolated nodes.

For instance in the case of G11 the prime with value 2 is isolated:

In [73]:
G11.has_node(2)

False